In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import transforms
import numpy as np
import pandas as pd
from PIL import Image, ImageEnhance
import argparse
import os
import copy
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from pprint import pprint
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
import cv2
from sklearn.naive_bayes import GaussianNB
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report

LABELS_Severity = {35: 0,
                   43: 0,
                   47: 1,
                   53: 1,
                   61: 2,
                   65: 2,
                   71: 2,
                   85: 2}


mean = (.1706)
std = (.2112)
normalize = transforms.Normalize(mean=mean, std=std)
train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    normalize,
])
test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    normalize,
])


In [49]:
class OCTDataset(Dataset):
    def __init__(self, annot=None, subset='train', transform=None, device='cpu'):
        if subset == 'train':
            self.annot = pd.read_csv("/content/drive/MyDrive/FML_Project/df_prime_train.csv")                      
        elif subset == 'test':
            self.annot = pd.read_csv("/content/drive/MyDrive/FML_Project/df_prime_test.csv")

        # Extract "Patient_ID" and "Week_Num" columns
        self.patient_ids = self.annot["Patient_ID"]
        self.week_nums = self.annot["Week_Num"]
        self.patient_ids = self.annot["Patient_ID"]
        self.annot['Severity_Label'] = [LABELS_Severity[drss] for drss in copy.deepcopy(self.annot['DRSS'].values)]
        self.drss_class = self.annot['Severity_Label']

        # Create unique pairs of values
        self.unique_pairs = set(zip(self.patient_ids, self.week_nums, self.drss_class))

        self.root = os.path.expanduser("/content/drive/MyDrive/FML_Project/")
        self.transform = transform
        self.nb_classes=len(np.unique(list(LABELS_Severity.values())))
        self.path_list = self.annot['File_Path'].values

        self._labels = [pair[2] for pair in self.unique_pairs]
        assert len(self.unique_pairs) == len(self._labels)
        
        max_samples = int(len(self._labels)) #32 #int(len(self._labels)/2)
        self.max_samples = max_samples
        self.device = device
        
    def __getitem__(self, index):
        # Get the Patient_ID and Week_Num from the indexed element in unique_pairs
        patient_id, week_num, target = list(self.unique_pairs)[index]

        # Filter the annot DataFrame to select rows that match the Patient_ID and Week_Num
        filtered_df = self.annot[(self.annot['Patient_ID'] == patient_id) & (self.annot['Week_Num'] == week_num)]

        # Extract the file paths from the filtered DataFrame and return them as a list
        file_paths = [self.root + file_path for file_path in filtered_df['File_Path'].values.tolist()]
        image_path = os.path.dirname(file_paths[0])+"/fused_image.jpg"
        # image_path = os.path.dirname(file_paths[0])+"/cn_final.jpg"
        # image_path = os.path.dirname(file_paths[0])+"/ab_final.png"
        # image_path = os.path.dirname(file_paths[0])+"/grid_image.jpg"
        # image_path = os.path.dirname(file_paths[0])+"/grid_image_canny.jpg"

        img = Image.open(image_path)
        img_gray = img.convert("L")

        if self.transform is not None:
            img_sharpened = self.transform(img_gray)
        
        return img_sharpened, target

    def __len__(self):
        if self.max_samples is not None:
            return min(len(self._labels), self.max_samples)
        else:
            return len(self._labels)    

In [50]:
#set up the device (GPU or CPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Found device:', device)

trainset = OCTDataset(subset='train', transform=train_transform, device=device)

#define the hyperparameters
batch_size = 32

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
print('Train and Test loader complete')

Found device: cuda:0
Train and Test loader complete


In [52]:
# Initialize the Gaussian Naive Bayes classifier
clf = GaussianNB()

# Train the classifier
train_features, train_labels = [], []
for i, (inputs, labels) in enumerate(trainloader):
    train_features.extend(inputs.view(inputs.shape[0], -1).numpy())
    train_labels.extend(labels.numpy())

train_features, train_labels = np.array(train_features), np.array(train_labels)
clf.fit(train_features, train_labels)

GaussianNB()

In [53]:
testset = OCTDataset(subset='test', transform=test_transform, device=device)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=4)
print(len(testloader))

# Test the classifier
test_features, test_labels = [], []
for inputs, labels in testloader:
    test_features.extend(inputs.view(inputs.shape[0], -1).numpy())
    test_labels.extend(labels.numpy())
test_features, test_labels = np.array(test_features), np.array(test_labels)

predicted_labels = clf.predict(test_features)

balanced_accuracy = balanced_accuracy_score(test_labels, predicted_labels)
f1 = f1_score(test_labels, predicted_labels, average='weighted')  # Use 'weighted' since we have imbalanced classes

print(f'Balanced accuracy: {balanced_accuracy:.4f}')
print(f'F1 score: {f1:.4f}')

6


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Balanced accuracy: 0.4955
F1 score: 0.4967


In [54]:
# Calculate the confusion matrix
cm = confusion_matrix(test_labels, predicted_labels)

# Calculate the sensitivity (recall) for each class
sensitivity = np.diag(cm) / np.sum(cm, axis=1)

# Calculate the specificity for each class
specificity = np.zeros(3)
for i in range(3):
    tp_fp = np.sum(cm[i, :])
    tp_fn = np.sum(cm[:, i])
    tp = cm[i, i]
    tn = np.sum(cm) - tp_fp - tp_fn + tp
    specificity[i] = tn / (tn + tp_fp - tp)

# Calculate the separability (harmonic mean of sensitivity and specificity)
separability = 2 * sensitivity * specificity / (sensitivity + specificity)

print("\nSensitivity (Recall):")
print("Class 0:", sensitivity[0])
print("Class 1:", sensitivity[1])
print("Class 2:", sensitivity[2])

print("\nSeparability:")
print("Class 0:", separability[0])
print("Class 1:", separability[1])
print("Class 2:", separability[2])


Sensitivity (Recall):
Class 0: 0.23076923076923078
Class 1: 0.675
Class 2: 0.5806451612903226

Separability:
Class 0: 0.34615384615384615
Class 1: 0.6450511945392491
Class 2: 0.7054012608621572
